In [1]:
#default_exp utils.math.box_ops

In [2]:
#export
import torch
import logging


In [3]:
#export
logger = logging.getLogger(__name__)

In [4]:
#export
def convert_boxes_to_2d_corners(boxes: torch.Tensor, batch_dim: bool = True) -> torch.Tensor:
    """Converts center representation to bounding corners
    :param boxes: tensor(batch_size, nbr_boxes, nb_attributes)
    :param batch_dim: Boolean if batch dim is used when converting

    :returns: tensor(batch_size, max_nbr_pred_boxes, 4)
                with last dimension being (x_min, y_min, x_max, y_max)
    """

    if batch_dim:
        xy = boxes[:,:,:2]
        lw = torch.stack((boxes[:,:,5], boxes[:,:,4]), dim=2)
        dim = 2
    else:
        xy = boxes[:,:2]
        lw = torch.stack((boxes[:,5], boxes[:,4]), dim=2)
        dim = 1

    xy_min = xy - 0.5 * lw
    xy_max = xy + 0.5 * lw

    return torch.cat([xy_min, xy_max], dim=dim)

In [5]:
#export
def convert_boxes_to_3d_corners(boxes: torch.Tensor, batch_dim: bool = True) -> torch.Tensor:
    """
    Converts center representation to bounding corners. According to Kitti conventions, when calculating
    z_min, 0 is subtracted and h is added to reach z_max.

    :param boxes: Tensor(nbr_batches, nbr_boxes, 7) with latest dim being (x,y,z,h,w,l,theta)
    :param batch_dim: Boolean if batch dimension is used when converting.

    :returns: Tensor(nbr_batches, nbr_boxes, 6)
                with latest dim being (x_min, y_min, z_min, x_max, y_max, z_max)
    """
    if batch_dim:
        xyz = boxes[:,:,:3]
        lw = torch.stack((boxes[:,:,5], boxes[:,:,4]), dim=2)
        h = boxes[:,:,3].unsqueeze(2)
        dim = 2
    else:
        xyz = boxes[:,:3]
        lw = torch.stack((boxes[:,5], boxes[:,4]), dim=1)
        h = boxes[:,3].unsqueeze(1)
        dim = 1

    zeros = torch.zeros_like(h, device="cuda:0")
    xyz_min = xyz - torch.cat((0.5 * lw, zeros), dim=dim)
    xyz_max = xyz + torch.cat((0.5 * lw, h), dim=dim)

    return torch.cat([xyz_min, xyz_max], dim=dim)


In [6]:
tens1 = torch.rand((2, 5, 7), device=torch.device("cuda"))
tens2 = torch.rand((5, 7), device=torch.device("cuda"))
print(tens1, tens1.shape)
out1 = convert_boxes_to_3d_corners(tens1, batch_dim=True)
print(out1, out1.shape)
out2 = convert_boxes_to_3d_corners(tens2, batch_dim=False)

out2, out2.shape


tensor([[[0.5994, 0.6235, 0.1382, 0.5332, 0.2336, 0.5083, 0.0521],
         [0.1153, 0.8175, 0.0230, 0.2062, 0.3402, 0.0817, 0.2251],
         [0.6932, 0.0083, 0.5972, 0.8764, 0.0683, 0.0276, 0.4219],
         [0.2351, 0.7263, 0.2560, 0.2200, 0.8178, 0.3944, 0.0321],
         [0.7057, 0.3701, 0.5154, 0.4128, 0.9739, 0.2762, 0.1336]],

        [[0.4584, 0.9925, 0.5483, 0.6735, 0.4109, 0.4803, 0.7455],
         [0.4367, 0.1756, 0.1170, 0.3456, 0.4874, 0.9136, 0.9629],
         [0.7565, 0.1352, 0.3870, 0.0571, 0.4442, 0.3100, 0.9955],
         [0.1332, 0.5678, 0.9045, 0.5601, 0.2069, 0.5565, 0.4605],
         [0.5226, 0.0381, 0.9285, 0.7056, 0.8407, 0.7728, 0.0536]]],
       device='cuda:0') torch.Size([2, 5, 7])
tensor([[[ 0.3452,  0.5067,  0.1382,  0.8535,  0.7403,  0.6714],
         [ 0.0744,  0.6474,  0.0230,  0.1561,  0.9876,  0.2292],
         [ 0.6794, -0.0259,  0.5972,  0.7070,  0.0425,  1.4736],
         [ 0.0379,  0.3174,  0.2560,  0.4323,  1.1351,  0.4761],
         [ 0.5676, -

(tensor([[ 0.4858,  0.2053,  0.1950,  1.4665,  1.0767,  0.3175],
         [ 0.9327,  0.7246,  0.9220,  1.0543,  1.0271,  1.4350],
         [ 0.0180,  0.2924,  0.2576,  0.3644,  1.1814,  0.9629],
         [ 0.2526, -0.4521,  0.2423,  0.3528,  0.5112,  0.7205],
         [ 0.1033,  0.0934,  0.9031,  0.2518,  0.5868,  1.5131]],
        device='cuda:0'),
 torch.Size([5, 6]))

In [7]:
#export
def bb_tensor_to_velodyne_coords(bb_cam: torch.Tensor):
    xc, yc, zc, box_dims = bb_cam[:,0], bb_cam[:, 1], bb_cam[:, 2], bb_cam[:, 3:]

    xv = zc
    yv = -1 * xc
    zv = -1 * yc

    return torch.cat((xv.unsqueeze(1), yv.unsqueeze(1), zv.unsqueeze(1), box_dims), dim=1)


In [8]:
#export
def convert_vel_to_cam_coords(point: torch.Tensor, batch_dim: bool = True) -> torch.Tensor:
    """
    Converts from velodyne coordinates to
    """
    if batch_dim:
        # TODO: implement batch_dim option
        raise NotImplementedError("batch_dim option not yet implemented.")



In [9]:
#export
def vel_point_to_left_cam_point(y: torch.Tensor, calib_file: str) -> torch.Tensor:
    """
     Convert vel point to left color image:
     x = P2 * R0_rect * Tr_velo_to_cam * y

     :returns: Tensor(4) with x,y,z,1 in image coordinates and system
    """
    # load transform data from calib folder


    with open(calib_file, 'r') as f:
        lines = f.readlines()
        P2_l = [float(x) for x in lines[2].split(" ")[1:]]
        R0_l = [float(x) for x in lines[4].split(" ")[1:]]
        Tr_l = [float(x) for x in lines[5].split(" ")[1:]]

    P2 = torch.tensor([[P2_l[0], P2_l[1], P2_l[2], P2_l[3]],
                       [P2_l[4], P2_l[5], P2_l[6], P2_l[7]],
                       [P2_l[8], P2_l[9], P2_l[10], P2_l[11]]], device=torch.device("cuda"))
    R0 = torch.tensor([[R0_l[0], R0_l[1], R0_l[2], 0],
                       [R0_l[3], R0_l[4], R0_l[5], 0],
                       [R0_l[6], R0_l[7], R0_l[8], 0],
                       [0,       0,       0,       1]], device=torch.device("cuda"))
    Tr = torch.tensor([[Tr_l[0], Tr_l[1], Tr_l[2],  Tr_l[3]],
                       [Tr_l[4], Tr_l[5], Tr_l[6],  Tr_l[7]],
                       [Tr_l[8], Tr_l[9], Tr_l[10], Tr_l[11]],
                       [0,       0,       0,        1]], device=torch.device("cuda"))

    return P2 @ R0 @ Tr @ y

In [20]:
calib_loc = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/calib/000000.txt"
y = torch.FloatTensor([0, 0, 0, 1])

vel_point_to_left_cam_point(y, calib_loc)

tensor([ 102.5686,    0.3276, -170.9427])